In [ ]:
# coding: utf-8

# In[ ]:

import pandas as pd
import os
import requests
import exchanges
import time
from multiprocessing.pool import ThreadPool
import pkgutil

exchange_list = []
prefix = exchanges.__name__ + "."
for importer, modname, ispkg in pkgutil.iter_modules(exchanges.__path__,prefix):
    print ("Found submodule %s (is a package: %s)"% (modname, ispkg))
    module = __import__(modname, fromlist="exchanges")
    exchange_list.append(module)




def getRealtimeData(symbol):
    results = []
    pool = ThreadPool()
    for exchange in exchange_list:
        results.append(pool.apply_async(exchange.getUSDPrice, args = (symbol,)))

    price_list = [r.get() for r in results if r is not None and r.get() is not None]
    pool.close()
    pool.join()
    
    if not price_list:
        return None
    df = pd.DataFrame(price_list)
    print(df)
    df = df[['source','bid','ask','trade']]
    df.columns=['Exchange','BestBid','BestAsk','Last Trade Price']
    return df



symbols = {'BTC':'Bitcoin','ETH':'Ethereum','BCH':'Bitcoin Cash','LTC':'Litecoin',
           'EOS':'EOS','XRP':'Ripple','STEEM':'Steem','BSV':'Bitcoin SV','TRX':'TRON',
           'ADA':'Cardano','MIOTA':'IOTA','XMR':'Monero','BNB':'Binance Coin',
           'DASH':'Dash','XEM':'NEM','ETC':'Ethereum Classic','NEO':'NEO','WAVES':'Waves','ZEC':'Zcash'}

for symbol in symbols:
    time3 = time.time()
    getRealtimeData(symbol)
    time4 = time.time()
    print ("time spent:",symbol,str(time4-time3))